In [1]:
import os
import pandas as pd
import numpy as np

import pyreadr

#import dask_mpi as dm
from dask.distributed import Client, progress
import dask

from datetime import datetime
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import joblib

In [2]:
!pwd

/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/07_PyCode


In [3]:
DP02_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/"
DP02_result_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/08_PyResults/"

In [2]:
DP02_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/"
DP02_result_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/08_PyResults/"

In [15]:
client = Client(n_workers=2, nthreads=1)

In [16]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58907,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:58914,Total threads: 5
Dashboard: http://127.0.0.1:58917/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:58910,


In [6]:
dataset = pyreadr.read_r(DP02_location + "02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds")
dataset = dataset[None]
y = np.array(dataset.iloc[:, 0:1].values.flatten(), dtype='float64')
X = np.array(dataset.iloc[:, 1:50], dtype='float64')

In [7]:
model = RandomForestRegressor(n_estimators=1000, oob_score=True, 
                               random_state=1, max_features = 9, n_jobs=-1)
with joblib.parallel_backend("dask"): model.fit(X, y)

In [8]:
# SHAP
import dalex as dx
model_rf_exp = dx.Explainer(model, X, y, label = "RF Pipeline")

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 89273 rows 49 cols
  -> target variable   : 89273 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : RF Pipeline
  -> predict function  : <function yhat_default at 0x7faa5da158b0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> predicted values  : min = 1.18, mean = 24.3, max = 36.0
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)


/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> residuals         : min = -11.2, mean = 0.0166, max = 7.13
  -> model_info        : package sklearn

A new explainer has been created!


In [17]:
X_scattered = client.scatter(X)
model_rf_exp_scattered = client.scatter(model_rf_exp)

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 11.42 GiB -- Worker memory limit: 16.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 12.11 GiB -- Worker memory limit: 16.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 12.74 GiB -- Worker memory limit: 16.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS

In [10]:
def singleSHAPprocess(obs_num, X, model_rf_exp):
    test_obs = X[obs_num:obs_num+1,:]
    shap_test = model_rf_exp.predict_parts(test_obs, type = 'shap', 
                                           B = 20, N = 1000)
    result = shap_test.result[shap_test.result.B == 0]
    result = result[['contribution', 'variable_name']]
    result = result.transpose()
    result = result.rename(columns=result.iloc[1])
    result = result.drop(['variable_name'], axis=0)
    result = result.reset_index(drop=True)
    return result

In [18]:
%%time
results = []
for obs_num in list(range(1000)):
    results.append(dask.delayed(singleSHAPprocess)(obs_num, X_scattered, model_rf_exp_scattered))

CPU times: user 47.2 ms, sys: 8.49 ms, total: 55.7 ms
Wall time: 53 ms


In [19]:
%%time
start = datetime.now()
test = dask.compute(*results) 
end = datetime.now()

distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 12.90 GiB -- Worker memory limit: 16.00 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 13.12 GiB -- Worker memory limit: 16.00 GiB
distributed.worker - WARNING - Worker is at 49% memory usage. Resuming worker. Process memory: 7.99 GiB -- Worker memory limit: 16.00 GiB
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
distribute

IndexError: positional indexers are out-of-bounds

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

...

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

In [45]:
%%time
test[999]

Wall time: 0 ns


,18,14,30,10,12,13,23,36,15,48,...,27,6,8,7,5,21,26,4,20,34
0,-0.901555,0.528859,-0.500492,0.484137,-0.402003,0.390392,0.345219,-0.219081,0.185294,-0.11907,...,0.002924,-0.002518,0.00205,0.00163,0.001456,0.00125,0.00055,0.000494,0.0001,0.0


In [39]:
results

[Delayed('singleSHAPprocess-e0910916-34e4-48ec-912b-08e0b00bd092'),
 Delayed('singleSHAPprocess-ff91032e-7a4b-4c2f-92c3-dc5030e54bd6'),
 Delayed('singleSHAPprocess-3dc74a2d-ce04-476c-befb-63723b61fc0c'),
 Delayed('singleSHAPprocess-6ed950fd-5ebd-485c-af84-5f798cfff8b4'),
 Delayed('singleSHAPprocess-c986087a-a481-4778-9084-7731809c26d3'),
 Delayed('singleSHAPprocess-97e50648-10e7-454a-b030-1f936b55fad7'),
 Delayed('singleSHAPprocess-3f9f8b5e-2f28-4f83-a5ec-95fd979ba90b'),
 Delayed('singleSHAPprocess-528e5754-f91c-4484-80de-3dddc0b24488'),
 Delayed('singleSHAPprocess-02c5da84-867c-4a8d-bde0-84085c9bf3ee'),
 Delayed('singleSHAPprocess-bb2e362f-c380-44c1-97e1-b196fa7a1f26'),
 Delayed('singleSHAPprocess-b0e8d820-5b3f-4920-b33f-59f29fc65afb'),
 Delayed('singleSHAPprocess-9d8596bc-791a-45a5-8e63-81c2982b8dd1'),
 Delayed('singleSHAPprocess-4685be39-700a-4854-bb4b-061edaaee009'),
 Delayed('singleSHAPprocess-364e2d01-0c38-4fb9-85ca-5cb5754aed5b'),
 Delayed('singleSHAPprocess-ad63becd-72a5-4307-b

In [71]:
list(range(8))

[0, 1, 2, 3, 4, 5, 6, 7]

In [20]:
client.close()

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted wi

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
distributed.nanny - WARNING - Worker process still alive after 3.999998474121094 seconds, killing


In [25]:
%%time
results_bag = joblib.Parallel(n_jobs=3, verbose=10000, backend = 'threading')(
    joblib.delayed(singleSHAPprocess)(int(obs_num), X, model_rf_exp)
    for obs_num in list(range(20)))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

...

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 